In [7]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files

class History(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        print ("Epoch End callback...")
        self.validation_losses.append(logs.get('val_loss'))
        self.validation_accuracy.append(logs.get('val_acc'))
        self.training_losses.append(logs.get('loss'))
        self.training_accuracy.append(logs.get('acc'))
    def on_train_begin(self, logs={}):
        self.validation_losses = []
        self.training_losses = []
        self.validation_accuracy = []
        self.training_accuracy = []
      
        
batch_size = 128
num_classes = 10
epochs = 12


file_handler = open("test_loss_and_accuracy.txt","w") 


##################### Data setup #####################################
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)




total_number_of_tests = 7

##################### Parameters setup #####################################
def set_test_params(test_id):
  if (test_id == 0):
    print("Test 0 : Run the code with the origianl parameter configuration")
    L1_kernel_size = (3,3)
    L3_kernel_size = (3,3)
    L1_nof_filters = 6
    L3_nof_filters = 16
  elif (test_id == 1): 
    print("Test 1 : Change the kernel size of the first conv2d layer to 5x5")
    L1_kernel_size = (5,5)
    L3_kernel_size = (3,3)
    L1_nof_filters = 6
    L3_nof_filters = 16
  elif (test_id == 2):
    print("Test 2 : Change the kernel size of the first conv2d layer to 5x5")
    L1_kernel_size = (5,5)
    L3_kernel_size = (5,5)
    L1_nof_filters = 6
    L3_nof_filters = 16
  elif (test_id == 3):
    print("Test 3 : Change the number of output filter 16 in the first conv2d and to 6 in the second conv2d")
    L1_kernel_size = (3,3)
    L3_kernel_size = (3,3)
    L1_nof_filters = 16
    L3_nof_filters = 6
  elif (test_id == 4 ):  
    print("Test 4 : Change the number of output filter 6 in the first conv2d and to 6 in the second conv2d")
    L1_kernel_size = (3,3)
    L3_kernel_size = (3,3)
    L1_nof_filters = 6
    L3_nof_filters = 6
  elif (test_id == 5):
    print("Test 5 : Change the number of output filter 16 in the first conv2d and to 16 in the second conv2d")
    L1_kernel_size = (3,3)
    L3_kernel_size = (3,3)
    L1_nof_filters = 16
    L3_nof_filters = 16
  elif (test_id == 6):
    print("Test 6 : Change the number of output filter 6 in the first conv2d and to 8 in the second conv2d")
    L1_kernel_size = (3,3)
    L3_kernel_size = (3,3)
    L1_nof_filters = 6
    L3_nof_filters = 8
    
  return L1_kernel_size,L3_kernel_size,L1_nof_filters,L3_nof_filters


##################### Setup, Run model and Log results #####################################
for test_id in range (0,total_number_of_tests):
  
  L1_kernel_size,L3_kernel_size,L1_nof_filters,L3_nof_filters = set_test_params(test_id)
  
  model = Sequential()
  model.add(Conv2D(L1_nof_filters, kernel_size=L1_kernel_size,
                   activation='relu',
                   input_shape=input_shape))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(L3_nof_filters, kernel_size=L3_kernel_size, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(120, activation='relu'))
  model.add(Dense(84, activation='relu'))

  model.add(Dense(num_classes, activation='softmax'))

  # https://keras.io/optimizers/ 
  model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0),
                metrics=['accuracy'])

  history = History()
  
  model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            validation_data=(x_test, y_test),callbacks=[history])

  score = model.evaluate(x_test, y_test, verbose=1)
  
   
  print('Test loss:', score[0])
  print('Test accuracy:', score[1])


  # Save test loss and accuracy results
  file_handler.write("Test params : \n")
  file_handler.write("\tL1_kernel_size = "+ str(L1_kernel_size) + "\n")
  file_handler.write("\tL3_kernel_size= " + str(L3_kernel_size) + "\n")
  file_handler.write("\tL1_nof_filters = "+ str(L1_nof_filters) + "\n")
  file_handler.write("\tL3_nof_filters = "+ str(L3_nof_filters) + "\n")
  
  file_handler.write("Test id = "+ str(test_id) + "\n")
  file_handler.write("Test loss = " +str(score[0]) + "\n")
  file_handler.write("Test accuracy = "+ str(score[1]) + "\n")
  file_handler.write("\n\n")

  plt.title('Losses Vs Number of Epochs')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  val_label = "Validation"
  train_label = "Training"

  epoch_range = np.arange(1,epochs+1)
  fig1 = plt.figure(1)
  plt.plot (epoch_range,history.validation_losses, label = val_label)
  plt.legend()
  plt.plot (epoch_range,history.training_losses, label = train_label)
  plt.legend()
  figure_name = "losses_vs_epochs"+str(test_id)+".png"

  plt.savefig(figure_name, dpi=300, bbox_inches='tight')
  plt.close(fig1)
  
  fig2 = plt.figure(2)
  plt.title('Accuracy Vs Number of Epochs')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.plot (epoch_range,history.validation_accuracy, label = val_label)
  plt.legend()
  plt.plot (epoch_range,history.training_accuracy, label = train_label)
  plt.legend()
  figure_name = "accuracy_vs_epochs"+str(test_id)+".png"
  plt.savefig(figure_name, dpi=300, bbox_inches='tight')
  plt.close(fig2)
  
  model = []
  history = []
  
  


file_handler.close()

('x_train shape:', (60000, 28, 28, 1))
(60000, 'train samples')
(10000, 'test samples')
Test 0 : Run the code with the origianl parameter configuration


KeyboardInterrupt: ignored

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
